# NLLB-200-distilled-1.3B 번역 테스트

속도와 품질이 균형잡힌 1.3B 모델 테스트

라이센스 X

## 1. 설치 및 import

In [ ]:
# 필요 시 설치
# !pip install transformers sentencepiece torch

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import time

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 모델 로드

약 2.6GB 다운로드 (1-2분)

In [ ]:
model_name = "facebook/nllb-200-distilled-1.3B"

print(f"📥 모델 로딩: {model_name}")

device = 0 if torch.cuda.is_available() else -1

# Pipeline 방식 (가장 간단)
translator_en2ko = pipeline(
    'translation',
    model=model_name,
    device=device,
    src_lang="eng_Latn",
    tgt_lang="kor_Hang",
    max_length=512
)

translator_ko2en = pipeline(
    'translation',
    model=model_name,
    device=device,
    src_lang="kor_Hang",
    tgt_lang="eng_Latn",
    max_length=512
)

print(f"✅ 로드 완료! (device: {'GPU' if device == 0 else 'CPU'})")

## 3. GPU 모니터링 함수

In [4]:
def get_gpu_memory():
    if torch.cuda.is_available():
        return {
            'allocated': torch.cuda.memory_allocated() / 1024**2,
            'reserved': torch.cuda.memory_reserved() / 1024**2,
        }
    return None

def print_gpu():
    mem = get_gpu_memory()
    if mem:
        print(f"GPU 메모리: {mem['allocated']:.0f}MB (예약: {mem['reserved']:.0f}MB)")

print_gpu()

GPU 메모리: 10473MB (예약: 10496MB)


## 4. 기본 번역 테스트

In [5]:
# 영어 -> 한국어
text_en = "The manufacturing execution system monitors production in real-time."

print(f"원문: {text_en}")
result = translator_en2ko(text_en)[0]['translation_text']
print(f"번역: {result}")
print_gpu()

원문: The manufacturing execution system monitors production in real-time.
번역: 제조 실행 시스템은 생산을 실시간으로 모니터링합니다.
GPU 메모리: 10481MB (예약: 10542MB)


In [6]:
# 한국어 -> 영어
text_ko = "제조 실행 시스템이 실시간으로 생산을 모니터링합니다."

print(f"원문: {text_ko}")
result = translator_ko2en(text_ko)[0]['translation_text']
print(f"번역: {result}")
print_gpu()

원문: 제조 실행 시스템이 실시간으로 생산을 모니터링합니다.
번역: The manufacturing execution system monitors production in real time.
GPU 메모리: 10481MB (예약: 10542MB)


## 5. MES 용어 테스트

In [7]:
mes_sentences = [
    "The MES system tracks work orders and monitors OEE.",
    "A defect was found during quality inspection.",
    "Update the production schedule for next week.",
    "The first pass yield improved to 92 percent.",
    "Equipment maintenance is required for Line 3."
]

print("=" * 80)
print("MES 문장 번역 테스트")
print("=" * 80)

for sent in mes_sentences:
    result = translator_en2ko(sent)[0]['translation_text']
    print(f"\nEN: {sent}")
    print(f"KO: {result}")

MES 문장 번역 테스트

EN: The MES system tracks work orders and monitors OEE.
KO: MES 시스템은 작업 명령을 추적하고 OEE를 모니터링합니다.

EN: A defect was found during quality inspection.
KO: 품질 검사에서 결함이 발견되었습니다.

EN: Update the production schedule for next week.
KO: 다음 주 생산 일정을 업데이트합니다.

EN: The first pass yield improved to 92 percent.
KO: 첫 합격률은 92%로 향상되었습니다.

EN: Equipment maintenance is required for Line 3.
KO: 3호선에는 장비 유지보수가 필요합니다.


## 6. 한국어 -> 영어

In [8]:
korean_sentences = [
    "제조실행시스템에서 작업지시서를 확인하세요.",
    "설비종합효율이 85퍼센트입니다.",
    "품질검사 중 불량이 발견되었습니다.",
    "생산 라인 3번의 설비 점검이 필요합니다."
]

print("=" * 80)
print("한국어 -> 영어 번역")
print("=" * 80)

for sent in korean_sentences:
    result = translator_ko2en(sent)[0]['translation_text']
    print(f"\nKO: {sent}")
    print(f"EN: {result}")

한국어 -> 영어 번역

KO: 제조실행시스템에서 작업지시서를 확인하세요.
EN: Check the manufacturing records in the manufacturing system.

KO: 설비종합효율이 85퍼센트입니다.
EN: The overall efficiency of the installation is 85 percent.

KO: 품질검사 중 불량이 발견되었습니다.
EN: We found a deficiency in quality control.

KO: 생산 라인 3번의 설비 점검이 필요합니다.
EN: We need three installation checks on the production line.


## 7. 속도 측정

In [9]:
def measure_speed(text, translator, iterations=5, warmup=1):
    # Warmup
    for _ in range(warmup):
        _ = translator(text)
    
    # 측정
    times = []
    for _ in range(iterations):
        start = time.time()
        result = translator(text)
        elapsed = time.time() - start
        times.append(elapsed)
    
    avg = sum(times) / len(times)
    return {
        'avg': avg,
        'min': min(times),
        'max': max(times),
        'result': result[0]['translation_text']
    }

print("=" * 80)
print("속도 측정 (5회 반복)")
print("=" * 80)

속도 측정 (5회 반복)


In [10]:
# 짧은 문장
short_text = "Check OEE dashboard."
result = measure_speed(short_text, translator_en2ko)

print(f"\n[짧은 문장] {short_text}")
print(f"평균: {result['avg']:.3f}초")
print(f"최소/최대: {result['min']:.3f}초 / {result['max']:.3f}초")
print(f"번역: {result['result']}")
print_gpu()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



[짧은 문장] Check OEE dashboard.
평균: 0.205초
최소/최대: 0.201초 / 0.211초
번역: OEE 대시보드를 확인해
GPU 메모리: 10481MB (예약: 10546MB)


In [11]:
# 중간 길이 문장
medium_text = "The MES system monitors production efficiency and tracks work orders in real-time."
result = measure_speed(medium_text, translator_en2ko)

print(f"\n[중간 문장] {medium_text}")
print(f"평균: {result['avg']:.3f}초")
print(f"최소/최대: {result['min']:.3f}초 / {result['max']:.3f}초")
print(f"번역: {result['result']}")
print_gpu()


[중간 문장] The MES system monitors production efficiency and tracks work orders in real-time.
평균: 0.345초
최소/최대: 0.338초 / 0.353초
번역: MES 시스템은 생산 효율성을 모니터링하고 작업 주문을 실시간으로 추적합니다.
GPU 메모리: 10481MB (예약: 10552MB)


In [12]:
# 긴 문장
long_text = """
The Manufacturing Execution System provides real-time information about production activities 
across the enterprise, enabling manufacturers to improve operational efficiency, quality control, 
and overall equipment effectiveness through data-driven decision making.
""".strip()

result = measure_speed(long_text, translator_en2ko, iterations=3)

print(f"\n[긴 문장] {len(long_text)} 글자")
print(f"평균: {result['avg']:.3f}초")
print(f"최소/최대: {result['min']:.3f}초 / {result['max']:.3f}초")
print(f"글자/초: {len(long_text)/result['avg']:.1f}")
print(f"\n번역: {result['result']}")
print_gpu()


[긴 문장] 266 글자
평균: 0.696초
최소/최대: 0.694초 / 0.697초
글자/초: 382.3

번역: 제조 실행 시스템은 기업 전체의 생산 활동에 대한 실시간 정보를 제공하여 제조업체가 데이터 기반 의사결정을 통해 운영 효율성, 품질 관리 및 전체 장비 효과를 향상시킬 수 있습니다.
GPU 메모리: 10481MB (예약: 10604MB)


## 8. 배치 처리 테스트

In [13]:
batch_texts = [
    "Check equipment status",
    "Update work order",
    "Review quality report",
    "Monitor production line",
    "Check inventory levels"
]

print(f"\n배치 처리: {len(batch_texts)}개 문장")

# 개별 처리
start = time.time()
for text in batch_texts:
    _ = translator_en2ko(text)
individual_time = time.time() - start

print(f"개별 처리: {individual_time:.3f}초 ({individual_time/len(batch_texts):.3f}초/문장)")
print_gpu()


배치 처리: 5개 문장
개별 처리: 0.885초 (0.177초/문장)
GPU 메모리: 10481MB (예약: 10604MB)


## 9. 품질 평가 (주관적)

In [14]:
quality_test = [
    ("The system detected an anomaly in sensor data.", "자연스러운 문장인가?"),
    ("Please escalate this issue to management.", "전문 용어 번역이 정확한가?"),
    ("The yield rate has been consistently above target.", "문맥이 유지되는가?"),
]

print("=" * 80)
print("품질 평가 (직접 확인 필요)")
print("=" * 80)

for text, question in quality_test:
    result = translator_en2ko(text)[0]['translation_text']
    print(f"\n원문: {text}")
    print(f"번역: {result}")
    print(f"평가: {question}")
    print("-" * 40)

품질 평가 (직접 확인 필요)

원문: The system detected an anomaly in sensor data.
번역: 시스템이 센서 데이터에 이상성을 감지했습니다.
평가: 자연스러운 문장인가?
----------------------------------------

원문: Please escalate this issue to management.
번역: 이 문제를 경영진에게 전달해 주세요.
평가: 전문 용어 번역이 정확한가?
----------------------------------------

원문: The yield rate has been consistently above target.
번역: 수익률은 지속적으로 목표보다 높았습니다.
평가: 문맥이 유지되는가?
----------------------------------------


## 10. 최종 요약

In [15]:
print("\n" + "=" * 80)
print("NLLB-200-distilled-1.3B 테스트 요약")
print("=" * 80)

print(f"\n모델: facebook/nllb-200-distilled-1.3B")
print(f"크기: ~2.6GB")
print(f"디바이스: {'GPU' if device == 0 else 'CPU'}")

if torch.cuda.is_available():
    mem = get_gpu_memory()
    print(f"GPU 메모리: {mem['allocated']:.0f}MB / {mem['reserved']:.0f}MB")

print("\n장점:")
print("  ✅ 빠른 속도 (1-2초/문장)")
print("  ✅ 적당한 메모리 (~5GB VRAM)")
print("  ✅ 200개 언어 지원")
print("  ✅ 우수한 번역 품질")

print("\n단점:")
print("  ⚠️ CC-BY-NC 라이센스 (연구용)")
print("  ⚠️ Glossary 미지원")

print("\n추천:")
print("  💡 MES 내부 사용에 최적")
print("  💡 속도와 품질의 균형")
print("  💡 실시간 처리 가능")

print("\n" + "=" * 80)


NLLB-200-distilled-1.3B 테스트 요약

모델: facebook/nllb-200-distilled-1.3B
크기: ~2.6GB
디바이스: GPU
GPU 메모리: 10481MB / 10604MB

장점:
  ✅ 빠른 속도 (1-2초/문장)
  ✅ 적당한 메모리 (~5GB VRAM)
  ✅ 200개 언어 지원
  ✅ 우수한 번역 품질

단점:
  ⚠️ CC-BY-NC 라이센스 (연구용)
  ⚠️ Glossary 미지원

추천:
  💡 MES 내부 사용에 최적
  💡 속도와 품질의 균형
  💡 실시간 처리 가능



## 📝 결론

### 비교: Hunyuan-MT-7B vs NLLB-1.3B

| 항목 | Hunyuan-MT-7B | NLLB-1.3B |
|------|---------------|------------|
| **크기** | 14GB | 2.6GB |
| **속도** | 2-5초 | 1-2초 |
| **메모리** | 16GB VRAM | 5GB VRAM |
| **품질** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ |
| **Glossary** | ✅ 지원 | ❌ 미지원 |
| **라이센스** | Apache 2.0 | CC-BY-NC |

### 추천:

- **빠른 속도 필요**: NLLB-1.3B ⭐⭐⭐⭐⭐
- **MES 용어 정확도**: Hunyuan-7B + Glossary ⭐⭐⭐⭐⭐
- **저사양 환경**: NLLB-600M 또는 NLLB-1.3B
- **외부 판매**: Hunyuan-7B (상업 라이센스)

### 다음 단계:

1. NLLB-600M도 테스트 (더 빠름)
2. 실제 MES 문서로 품질 비교
3. 최종 모델 선택
4. MCP 서버 구현